 ------------- Last modifying at 15 Jan, 2024 -----------------

# Unused

# Drive Mount

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import numpy as np
from sklearn.cluster import KMeans


root_folder = './'
#set hidden state of the lstm
hidden_state_of_lstm = 40
# learning_rate = 0.0002
learning_rate = 0.001
server_percentage = 0.5 # percent of data in server
# server_percentage_2 = 1.0 # percent of data in server
# user_percentage_2 = 1.0
isolated_users_percentage = 0.1  # Change this according to your requirement
# isolated_users_percentage = 0.0  # Change this according to your requirement
overlapping_percentage = 0  # Change this according to your requirement
cluster_head_count = 25 #10,20,30,40,50

save_file_path = "k_means_clustered_" + "hs_" + str(hidden_state_of_lstm) + "_lr_" + str(int(learning_rate * 10000)) + "_Sp_" + str(int(server_percentage * 100))+"_Iu_" + str(int(isolated_users_percentage * 100)) + "_Op_" + str(int(overlapping_percentage * 100)) + "_Cl_" + str(cluster_head_count)



In [ ]:

# new_int = np.random.randint(10)
# print(new_int)

In [3]:
# from tqdm import tqdm
import math
import pandas as pd
import numpy as np
import csv
import os
import sys
from datetime import datetime, timedelta

# Data Creation

## Unused

## Old Data Distribution

### Data Distribution based of Cross Mixation of a certain amount of data inside both the user data and server data

#### Old User data Generation

# Unused

#### Old Server Data Generation

### Creating testing data as 1st 3 months data and remaining data as training data

# New Data Distribution

## Data Distribution based on "Server data will have certain portion of data from each 77 region"

### first get the entire train portion

In [4]:
import pandas as pd
import numpy as np

# Load your CSV file into a DataFrame
# train_data = pd.read_csv('./training_crime_data_for_last_9_months.csv')
train_data = pd.read_csv('./Final_Training_Data_side.csv')

# Assuming 'timestamp' is the column name for timestamp and 'neighbourhood_id' is the column for neighbourhood_id
train_data['time'] = pd.to_datetime(train_data['time'], format='%Y-%m-%d %H:%M:%S')


# Now from the train data, give 100 percent data to both server and all the users, then get a certain cluster heads and assign all the crimes to those cluster heads

## New User Assigment to Crimes Code for New Experiments

In [6]:
# Step 2: Split train data into server and user based on beta percentage for each neighbourhood_id
# server_percentage = 0.25 
# server_percentage = 0.70 # percent of data in server

# Set the seed
seed_value = 46
np.random.seed(seed_value)

#rename the Community Area to neighbourhood_id
train_data.rename(columns={"Community Area": "neighbourhood_id"}, inplace=True)
# Group by neighbourhood_id
grouped_train_data = train_data.groupby('neighbourhood_id')


server_data = pd.DataFrame()  # To store the server data
user_data = pd.DataFrame()    # To store the user data

# Iterate through each group
for group_name, group_data in grouped_train_data:
    # Shuffle the rows within each group
    shuffled_group_data = group_data.sample(frac=1, random_state=42)  # Set a random_state for reproducibility
    
    # Split based on beta percentage
    server_rows = shuffled_group_data.head(int(server_percentage * len(shuffled_group_data)))
    user_rows = shuffled_group_data.tail(len(shuffled_group_data) - len(server_rows))

    # Concatenate the server and user data for each group
    server_data = pd.concat([server_data, server_rows])
    user_data = pd.concat([user_data, user_rows])

# Now, 'server_data' and 'user_data' contain the final server and user data respectively
# here the server and user data are both "train" data which are non overlapping
#TODO: we will assign uid to the user_data now
CHECKIN_PATH = './checkins_with_neighborhood_id_side.csv'
checkins = pd.read_csv(CHECKIN_PATH)
checkins.drop(['venueid', 'venue_category', 'count'], axis=1, inplace=True)
checkins['utctime'] = pd.to_datetime(checkins['utctime']).dt.tz_localize(None)
checkins.rename(columns={"utctime": "time"}, inplace=True)
checkins.sort_values(by=['time'], inplace=True)

# client_list_500 = list(range(1, 501))  # Creates a list from 1 to 500 (inclusive)
# np.random.shuffle(client_list_500)
def assign_uid_to_crime():
    for i in range(1,10):
        crimes_i = user_data[user_data['neighbourhood_id'] == i]
        checkins_i = checkins[checkins['neighbourhood_id'] == i]
        print(f"region:  {i}, crimes :{crimes_i.shape[0]}, checkins: {checkins_i.shape[0]}, uid: {checkins_i['userid'].unique().shape[0]}")
        if(checkins_i['userid'].unique().shape[0] > 0):
            # Shuffle the array to ensure randomness
            np.random.shuffle(checkins_i['userid'].unique())

            # Take the first cluster_head_count values
            unique_user_idssss = checkins_i['userid'].unique()
            # print('unique user iddss ',len(unique_user_idssss))
            actual_unique_user_length = len(unique_user_idssss)
            # print(f'len of actual uniq users for ${i} ',actual_unique_user_length)
            # selected_uids = set()
            index_count = 0
            for index, row in crimes_i.iterrows():
                # user_data.loc[index, 'uid'] = int(np.random.choice(checkins_i['userid']))
                # user_data.loc[index, 'uid'] = int(np.random.choice(checkins_i['userid'].unique()[:cluster_head_count]))
                # user_data.loc[index, 'uid'] = int(np.random.choice(20))
                # Ensure that we loop back to the beginning of the array if we reach its end
                uid = int(unique_user_idssss[index_count % actual_unique_user_length])
                # selected_uids.add(uid)
                user_data.loc[index, 'uid'] = uid
                index_count+=1
            
            # print(f'selected uid for region {i} is ',len(selected_uids))
        else :
            for index, row in crimes_i.iterrows():
                user_data.loc[index, 'uid'] = -1
        # unique_user_idssss = client_list_500[:cluster_head_count]
        # unique_user_idssss = checkins_i['userid'].unique()[:cluster_head_count]
        # actual_unique_user_length = len(unique_user_idssss)
        # for index, row in crimes_i.iterrows():
        #     uid = int(unique_user_idssss[index % actual_unique_user_length])
        #     user_data.loc[index, 'uid'] = uid
    print('uid assignment for all the 77 regions done,but there are -1 uid also so we have to replace them with those uids which have very low crime incidents')
    uid_row_count = user_data['uid'].value_counts().reset_index()
    uid_row_count.columns = ['uid', 'crime_count']

    # Sort the DataFrame by 'uid'
    uid_row_count = uid_row_count.sort_values(by='crime_count')


    # Get the top 1946 uid values
    top_uid_list = uid_row_count.head(1946)['uid'].tolist()
    # print('top uid list ',top_uid_list)

    # Read the DataFrame where uid == -1
    # df_uid_minus_one = user_data[user_data['uid'] == -1]

    # # Assign a randomly chosen uid from the top 1946 uid list
    # df_uid_minus_one['uid'] = np.random.choice(top_uid_list, size=len(df_uid_minus_one), replace=True)
    # df_uid_minus_one['uid'] = df_uid_minus_one['uid'].astype(int)
    
    # Use .loc to update the DataFrame where uid == -1
    df_uid_minus_one = user_data[user_data['uid'] == -1]
    user_data.loc[user_data['uid'] == -1, 'uid'] = np.random.choice(top_uid_list, size=len(df_uid_minus_one), replace=True)
    # user_data.loc[user_data['uid'] == -1, 'uid'] = np.random.choice(top_uid_list, size=len(df_uid_minus_one), replace=False)
    user_data['uid'] = user_data['uid'].astype(int)

    print('--------------- done with replacing -1 with the random uid which has very less data (random from bottom, around 1946 unique uids which has very very less crime data)')
    print("######### ------------- All Crimes have been assigned with a User ------------- ##################")
    
assign_uid_to_crime()
    


region:  1, crimes :6638, checkins: 14128, uid: 1278
region:  2, crimes :3493, checkins: 5256, uid: 578
region:  3, crimes :5658, checkins: 39150, uid: 2832
region:  4, crimes :18675, checkins: 28063, uid: 2787
region:  5, crimes :7730, checkins: 60565, uid: 5127
region:  6, crimes :9667, checkins: 7861, uid: 1242
region:  7, crimes :9318, checkins: 7000, uid: 1671
region:  8, crimes :4516, checkins: 1737, uid: 267
region:  9, crimes :8208, checkins: 3468, uid: 396
uid assignment for all the 77 regions done,but there are -1 uid also so we have to replace them with those uids which have very low crime incidents
--------------- done with replacing -1 with the random uid which has very less data (random from bottom, around 1946 unique uids which has very very less crime data)
######### ------------- All Crimes have been assigned with a User ------------- ##################


## Now Use K-Means Clustering to cluster these Users

In [10]:
def K_Means_Clustering(user_data):
    user_vectors = {}

    for region in range(1, 78):
        checkins_region = checkins[checkins['neighbourhood_id'] == region]
        
        if checkins_region.shape[0] > 0:
            user_ids = checkins_region['userid'].unique()
            for user_id in user_ids:
                user_vectors[user_id] = [0] * 9  # Initialize vector of size 9 with zeros

            crimes_region = user_data[user_data['neighbourhood_id'] == region]
            for user_id in user_ids:
                user_crimes = crimes_region[crimes_region['uid'] == user_id]
                for month in range(4, 13):  # From April (4) to December (12)
                    start_date = pd.Timestamp(year=2019, month=month, day=1)
                    if month == 12:
                        end_date = pd.Timestamp(year=2019, month=month, day=31, hour=23, minute=59, second=59)
                    else:
                        end_date = pd.Timestamp(year=2019, month=month+1, day=1) - pd.Timedelta(seconds=1)
                    
                    crime_count = user_crimes[(user_crimes['time'] >= start_date) & (user_crimes['time'] <= end_date)].shape[0]
                    user_vectors[user_id][month-4] = crime_count  # Fill the vector for the corresponding month
    

    # Convert user_vectors dictionary to a list of vectors
    user_ids = list(user_vectors.keys())
    vectors = list(user_vectors.values())

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=cluster_head_count, random_state=0).fit(vectors)

    # Create a dictionary to store the clusters
    clusters = {i: [] for i in range(cluster_head_count)}

    # Assign user_ids to their respective clusters
    for idx, label in enumerate(kmeans.labels_):
        clusters[label].append(user_ids[idx])

    # Print the clusters
    for cluster_id, users in clusters.items():
        print(f"Cluster {cluster_id}: {users}")


K_Means_Clustering(user_data)

Cluster 0: [456, 872, 605, 2639, 826, 1347, 4659, 625, 2198, 1924, 109, 380, 100, 984, 373, 4602, 943, 1894, 197, 26, 976, 1810, 1105, 1782, 740, 1106, 4137, 837, 683, 246, 2296, 428, 676, 1747, 354, 1234, 574, 75, 866, 983, 16, 972, 181, 52, 494, 519, 2911, 997, 538, 1572, 1149, 2588, 3246, 540, 304, 439, 99, 1678, 4069, 3083, 2325, 399, 765, 1790, 71, 5006, 303, 919, 2938, 320, 339, 505, 4066, 1178, 606, 572, 1027, 797, 6691, 1557, 846, 244, 1172, 315, 307, 1468, 260, 1935, 2834, 2810, 554, 587, 3595, 3684, 937, 143, 669, 1299, 938, 824, 791, 1254, 967, 969, 1388, 2245, 1379, 190, 1629, 2440, 2149, 1282, 1346, 1905, 3975, 660, 2203, 406, 1333, 704, 4142, 5103, 1173, 6163, 3253, 876, 328, 1261, 3352, 1101, 1426, 3032, 3131, 624, 1294, 514, 930, 2727, 440, 1481, 1300, 836, 2916, 4812, 1632, 2584, 4558, 1858, 2091, 1539, 381, 1524, 1820, 2293, 2377, 1703, 5073, 1487, 1516, 2869, 2267, 3039, 3069, 2472, 2981, 6334, 4107, 3266, 4156, 1821, 2745, 2447, 1588, 2580, 2966, 2281, 2000, 6478, 2

### ~~Now from the train data, seperate the server portion and user portion ( perviously done) ~~

In [ ]:

# # Step 2: Split train data into server and user based on beta percentage for each neighbourhood_id
# # server_percentage = 0.25
# # server_percentage = 0.70 # percent of data in server

# # Set the seed
# seed_value = 46
# np.random.seed(seed_value)

# #rename the Community Area to neighbourhood_id
# train_data.rename(columns={"Community Area": "neighbourhood_id"}, inplace=True)
# # Group by neighbourhood_id
# grouped_train_data = train_data.groupby('neighbourhood_id')

# server_data = pd.DataFrame()  # To store the server data
# user_data = pd.DataFrame()    # To store the user data

# # Iterate through each group
# for group_name, group_data in grouped_train_data:
#     # Shuffle the rows within each group
#     shuffled_group_data = group_data.sample(frac=1, random_state=42)  # Set a random_state for reproducibility

#     # Split based on beta percentage
#     server_rows = shuffled_group_data.head(int(server_percentage * len(shuffled_group_data)))
#     user_rows = shuffled_group_data.tail(len(shuffled_group_data) - len(server_rows))

#     # Concatenate the server and user data for each group
#     server_data = pd.concat([server_data, server_rows])
#     user_data = pd.concat([user_data, user_rows])

# # Now, 'server_data' and 'user_data' contain the final server and user data respectively
# # here the server and user data are both "train" data which are non overlapping
# #TODO: we will assign uid to the user_data now
# CHECKIN_PATH = './checkins_with_neighborhood_id.csv'
# checkins = pd.read_csv(CHECKIN_PATH)
# checkins.drop(['venueid', 'venue_category', 'count'], axis=1, inplace=True)
# checkins['utctime'] = pd.to_datetime(checkins['utctime']).dt.tz_localize(None)
# checkins.rename(columns={"utctime": "time"}, inplace=True)
# checkins.sort_values(by=['time'], inplace=True)

# # client_list_500 = list(range(1, 501))  # Creates a list from 1 to 500 (inclusive)
# # np.random.shuffle(client_list_500)
# def assign_uid_to_crime():
#     for i in range(1,78):
#         crimes_i = user_data[user_data['neighbourhood_id'] == i]
#         checkins_i = checkins[checkins['neighbourhood_id'] == i]
#         print(f"region:  {i}, crimes :{crimes_i.shape[0]}, checkins: {checkins_i.shape[0]}, uid: {checkins_i['userid'].unique().shape[0]}")
#         if(checkins_i['userid'].unique().shape[0] > 0):
#             # Shuffle the array to ensure randomness
#             np.random.shuffle(checkins_i['userid'].unique())

#             # Take the first cluster_head_count values
#             unique_user_idssss = checkins_i['userid'].unique()[:cluster_head_count]
#             # print('unique user iddss ',len(unique_user_idssss))
#             actual_unique_user_length = len(unique_user_idssss)
#             # print(f'len of actual uniq users for ${i} ',actual_unique_user_length)
#             # selected_uids = set()
#             index_count = 0
#             for index, row in crimes_i.iterrows():
#                 # user_data.loc[index, 'uid'] = int(np.random.choice(checkins_i['userid']))
#                 # user_data.loc[index, 'uid'] = int(np.random.choice(checkins_i['userid'].unique()[:cluster_head_count]))
#                 # user_data.loc[index, 'uid'] = int(np.random.choice(20))
#                 # Ensure that we loop back to the beginning of the array if we reach its end
#                 uid = int(unique_user_idssss[index_count % actual_unique_user_length])
#                 # selected_uids.add(uid)
#                 user_data.loc[index, 'uid'] = uid
#                 index_count+=1

#             # print(f'selected uid for region {i} is ',len(selected_uids))
#         else :
#             for index, row in crimes_i.iterrows():
#                 user_data.loc[index, 'uid'] = -1
#         # unique_user_idssss = client_list_500[:cluster_head_count]
#         # unique_user_idssss = checkins_i['userid'].unique()[:cluster_head_count]
#         # actual_unique_user_length = len(unique_user_idssss)
#         # for index, row in crimes_i.iterrows():
#         #     uid = int(unique_user_idssss[index % actual_unique_user_length])
#         #     user_data.loc[index, 'uid'] = uid
#     print('uid assignment for all the 77 regions done,but there are -1 uid also so we have to replace them with those uids which have very low crime incidents')
#     uid_row_count = user_data['uid'].value_counts().reset_index()
#     uid_row_count.columns = ['uid', 'crime_count']

#     # Sort the DataFrame by 'uid'
#     uid_row_count = uid_row_count.sort_values(by='crime_count')


#     # Get the top 1946 uid values
#     top_uid_list = uid_row_count.head(1946)['uid'].tolist()
#     # print('top uid list ',top_uid_list)

#     # Read the DataFrame where uid == -1
#     # df_uid_minus_one = user_data[user_data['uid'] == -1]

#     # # Assign a randomly chosen uid from the top 1946 uid list
#     # df_uid_minus_one['uid'] = np.random.choice(top_uid_list, size=len(df_uid_minus_one), replace=True)
#     # df_uid_minus_one['uid'] = df_uid_minus_one['uid'].astype(int)

#     # Use .loc to update the DataFrame where uid == -1
#     df_uid_minus_one = user_data[user_data['uid'] == -1]
#     user_data.loc[user_data['uid'] == -1, 'uid'] = np.random.choice(top_uid_list, size=len(df_uid_minus_one), replace=True)
#     # user_data.loc[user_data['uid'] == -1, 'uid'] = np.random.choice(top_uid_list, size=len(df_uid_minus_one), replace=False)
#     user_data['uid'] = user_data['uid'].astype(int)

#     print('--------------- done with replacing -1 with the random uid which has very less data (random from bottom, around 1946 unique uids which has very very less crime data)')




In [5]:

# Set the seed
seed_value = 46
np.random.seed(seed_value)
# assign_uid_to_crime()
print(user_data.head(3))
print('-------------- uid assignment done --------------------')

# ------------------ uid assign done -------------------------
# Step 3: Isolate a certain portion of uid (isolated_users_percentage) with no data shared with the server
# user_data['uid'] = user_data['uid'].astype(int)
unique_uids = user_data['uid'].unique()
isolated_uids = np.random.choice(unique_uids, size=int(isolated_users_percentage * len(unique_uids)), replace=False)
isolated_uids = isolated_uids.astype(int)
# isolated_user_data = user_data[user_data['uid'].isin(isolated_uids)]

# Step 4: Generate new overlapping user data

#make sure that the neighbourhood_id is integer
server_data['neighbourhood_id'] = server_data['neighbourhood_id'].astype(int)

new_overlapping_user_data = pd.DataFrame()

if(overlapping_percentage != 0):
    # Iterate through each group for server data
    for group_name, group_data in server_data.groupby('neighbourhood_id'):
        # Extract overlapping_percentage percentage of random server data
        extracted_group_data = group_data.sample(frac=overlapping_percentage, random_state=42)

        # Get the unique uid for the current group
        unique_uids_for_group = user_data[user_data['neighbourhood_id'] == group_name]['uid'].unique()

        # Assign a random uid from the remaining uids to each data point in the extracted group data
        remaining_uids = np.setdiff1d(unique_uids_for_group, isolated_uids)
        if remaining_uids.size != 0:
            extracted_group_data['uid'] = np.random.choice(remaining_uids, size=len(extracted_group_data), replace=True)
            extracted_group_data['uid'] = extracted_group_data['uid'].astype(int)

        # Concatenate with the new overlapping user data
        new_overlapping_user_data = pd.concat([new_overlapping_user_data, extracted_group_data])

    # Concatenate the new overlapping user data with the isolated user data to get the final user data
    user_data = pd.concat([user_data, new_overlapping_user_data])

# Now, 'server_data' and 'user_data' contain the final server and user data respectively
server_data.to_csv(f'./server_data_for_FED_AIST_{save_file_path}.csv', index=False)
user_data.to_csv(f'./crimes_with_uid_{save_file_path}.csv', index=False)
print('----------- user data and server data saved in csv -------------')

uid_row_count = user_data['uid'].value_counts().reset_index()
uid_row_count.columns = ['uid', 'crime_count']

# Sort the DataFrame by 'uid'
uid_row_count = uid_row_count.sort_values(by='crime_count', ascending=False)
# Store the results in a new CSV file
uid_row_count.to_csv(f'user_crimes_count_{save_file_path}.csv', index=False)
print('-------------user crime count also stored -------------------------')



--------------- done with putting uids of cluster heads to all the crime incidents of all the regions
                    time  IUCR        Primary Type  neighbourhood_id  \
1678 2019-08-03 01:00:00  1320     CRIMINAL DAMAGE                 1   
2325 2019-04-07 17:00:00  1130  DECEPTIVE PRACTICE                 1   
1757 2019-11-20 19:02:00  0320             ROBBERY                 1   

            lat        lon  TYPE_ID  uid  
1678  42.012523 -87.680949        2  290  
2325  42.006997 -87.675288        0   75  
1757  42.001065 -87.661254        4  101  
-------------- uid assignment done --------------------
----------- user data and server data saved in csv -------------
-------------user crime count also stored -------------------------


In [8]:
# getting the region wise crime list to sort which region has greater crime incidents
user_data_for_new_calc = pd.read_csv(f'./crimes_with_uid_{save_file_path}.csv')
region_crime_count = user_data_for_new_calc['neighbourhood_id'].value_counts().reset_index()
region_crime_count.columns = ['region_id', 'crime_count']

# Sort the DataFrame by 'uid'
region_crime_count = region_crime_count.sort_values(by='crime_count', ascending=False)
# Store the results in a new CSV file
region_crime_count.to_csv(f'region_crime_counts_{save_file_path}.csv', index=False)
print('-------------region crime count also stored -------------------------')

-------------region crime count also stored -------------------------


In [10]:
corresponding_uids_data = pd.read_csv(root_folder+f'/user_crimes_count_{save_file_path}.csv')
corresponding_uids = corresponding_uids_data['uid'].to_numpy()
print(len(corresponding_uids))

50


#### User Train Data Generation

In [11]:
# # now we will create the side dataset based on data/chicago/side_com_adj.txt file
'''from here we will need the rest of the codes,'''
# from tqdm.notebook import tqdm, trange
import shutil

crime_mapping = {
    'DECEPTIVE PRACTICE': 0,
    'THEFT': 1,
    'CRIMINAL DAMAGE': 2,
    'BATTERY': 3,
    'ROBBERY': 4,
    'ASSAULT': 5,
    'BURGLARY': 6,
    'NARCOTICS': 7
    }

side_neighbourhood_dict = {}
with open('./side_com_adj.txt', 'r') as f:
    # read the lines
    lines = f.readlines()
    # now here each line has 2 integers, 1st is side_id and 2nd is neighbourhood_id
    # create a dictionary, where key is neighbourhood_id and value is the side_id
    for line in lines:
        # split the line by space
        line = line.split()
        # get the side_id
        side_id = int(line[0])
        # get the neighbourhood_id
        neighbourhood_id = int(line[1])
        # modulus the side_id by 101
        side_id = side_id % 101
        # add the side_id to the dictionary
        side_neighbourhood_dict[neighbourhood_id-1] = side_id  # here 1 is reduced because id starts from 1, making it 0 based indexing

# START_DATE = datetime.strptime('2012-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# END_DATE = datetime.strptime('2013-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# START_DATE = datetime.strptime('01/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S')
# END_DATE = datetime.strptime('09/19/2019 07:59:59', '%m/%d/%Y %H:%M:%S')

START_DATE = datetime.strptime('04/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S')
END_DATE = datetime.strptime('12/31/2019 23:59:59', '%m/%d/%Y %H:%M:%S')
# read the data lat lon file
# user_crime_full_df = pd.read_csv('/content/drive/MyDrive/Undergrad_thesis_final/AIST_UCA/data_latlon_xy_with_neighbourhood_id.csv')
user_crime_full_df = pd.read_csv(f'./crimes_with_uid_{save_file_path}.csv')
user_crime_full_df['time'] = pd.to_datetime(user_crime_full_df['time'], format='%Y-%m-%d %H:%M:%S')
# find the unique user ids from this df
# unique_user_ids = user_crime_full_df['uid'].unique()
#from all the unique user ids, take 1000 user_ids that are not present in the corresponding uids
# Filter out 1000 uids that are not present in corr_uids
# filtered_uids_for_server_data = [uid for uid in unique_user_ids if uid not in corresponding_uids][:2000]
# get the rest uids for cross_mixing
# cross_mixing_uids = [uid for uid in unique_user_ids if uid not in filtered_uids_for_server_data and uid not in corresponding_uids]


#now get the dataframe where the uids are the cross_mixing_uids
# cross_mixing_df = user_crime_full_df[user_crime_full_df['uid'].isin(cross_mixing_uids)]

# print('unique user ids are ',len(unique_user_ids))
# sort this unique_user_ids in ascending order
# unique_user_ids.sort()
# taken_user_ids = 1015  # here many users are absent so we took 1015 to get approximately 1000 users
# unique_user_ids = unique_user_ids[:taken_user_ids]
# for each user id, create a directory in "../data/chicago2/" directory named that particular user id if the directory doesn't exist already
# for id in trange(len(corresponding_uids)):
for id in range(len(corresponding_uids)):
    user_id = corresponding_uids[id]
    # create the directory
    # os.makedirs('/data/chicago/user/' + str(user_id), exist_ok=True)
    # create a new df for each user id
    user_crime_df = user_crime_full_df[user_crime_full_df['uid'] == user_id]

    total_meighbors = 77
    # total_days = 260
    # total_days = 365  # calculted from START_DATE and END_DATE,not 260
    # total_days = 261  # calculted from START_DATE and END_DATE,not 365, (END_DATE - START_DATE)  (excludes the END_DATE)
    total_days = 275  # calculted from START_DATE and END_DATE,not 261, (END_DATE - START_DATE)  (includes the END_DATE)
    time_division_per_day = 6 # here each day is divided in 12 hrs slot, so each day has 2hrs slot
    hours_per_cell  = 4
    total_cols = total_days * time_division_per_day
    # now we have only first 8 hours (07:59:59) for the END_DATE, so we cannot take the whole day, rather we will take additional(8/hours_per_cell) = (8/4)= 2 columns extra
    # total_cols +=2
    # total number of row is total crimes in crime_mapping
    total_rows = len(crime_mapping)
    # create a 3d tensor of dimension total_regions X crimes X total_cols
    user_crime_tensor = np.zeros((total_meighbors, total_rows, total_cols))

    '''code for adding the side crimes'''
    # create a user side crime tensor of dimenstion 9 X crimes X total_cols
    user_side_crime_tensor = np.zeros((9, total_rows, total_cols))

    # Convert the time format to datetime
    # user_crime_df['time'] = pd.to_datetime(user_crime_df['time'], format='%Y-%m-%d %H:%M:%S')

    # run loop for each row
    for index, row in user_crime_df.iterrows():
        # get the neighbourhood id
        neighbourhood_id = row['neighbourhood_id']
        neighbourhood_id -=1 # for 0 based indexing
        # get the crime type id
        crime_type_id = row['TYPE_ID']
        # get the date
        date = row['time']
        # get the time
        time = date.time()
        # get the day
        day = date.date()
        # get the time division
        time_division = int(math.floor(time.hour / hours_per_cell))
        # get the day index
        day_index = (day - START_DATE.date()).days
        # get the time division index
        time_division_index = day_index * time_division_per_day + time_division
        # increment the value in the tensor
        user_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1   # for com_crime calculations
        cor_side_id = side_neighbourhood_dict[neighbourhood_id]
        user_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1   # for side crime calculations



    #mixing with user data started
    # random_sampling_percentage = 20
    #now take random_sampling_percentage of dataframe from crime_mixing_df
    # crime_mixing_df_for_current_user = cross_mixing_df.sample(frac=random_sampling_percentage/100)

    # crime_mixing_df_for_current_user['time'] = pd.to_datetime(crime_mixing_df_for_current_user['time'], format='%Y-%m-%d %H:%M:%S')

    # run loop for each row
    # for index, row in crime_mixing_df_for_current_user.iterrows():
    #     # get the neighbourhood id
    #     neighbourhood_id = row['neighbourhood_id']
    #     neighbourhood_id -=1 # for 0 based indexing
    #     # get the crime type id
    #     crime_type_id = row['TYPE_ID']
    #     # get the date
    #     date = row['time']
    #     # get the time
    #     time = date.time()
    #     # get the day
    #     day = date.date()
    #     # get the time division
    #     time_division = int(math.floor(time.hour / hours_per_cell))
    #     # get the day index
    #     day_index = (day - START_DATE.date()).days
    #     # get the time division index
    #     time_division_index = day_index * time_division_per_day + time_division
    #     # increment the value in the tensor
    #     user_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1
    #     cor_side_id = side_neighbourhood_dict[neighbourhood_id]
    #     user_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1

    # Iterate over the first dimension (total meighbors) and save each 2D slice as a text file
    output_directory = f'./toy_data/chicago/user_{save_file_path}/'+str(user_id)
    if os.path.exists(output_directory):
      shutil.rmtree(output_directory)

    os.makedirs(output_directory)

    for i in range(total_meighbors):
        filename = f"{output_directory}/r_{i}.txt"
        # Extract the 2D slice at index i
        slice_2d = user_crime_tensor[i, :, :]

        # Save the 2D slice as a text file
        np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")


    # now same for side data
    for i in range(9):
        filename = f"{output_directory}/s_{i}.txt"
        # Extract the 2D slice at index i
        slice_2d = user_side_crime_tensor[i, :, :]

        # Save the 2D slice as a text file
        np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")

    print('user id completed : ',user_id)




user id completed :  130
user id completed :  64
user id completed :  319
user id completed :  886
user id completed :  79
user id completed :  71
user id completed :  411
user id completed :  695
user id completed :  1210
user id completed :  75
user id completed :  0
user id completed :  538
user id completed :  12
user id completed :  16
user id completed :  127
user id completed :  116
user id completed :  1083
user id completed :  180
user id completed :  290
user id completed :  6433
user id completed :  198
user id completed :  394
user id completed :  101
user id completed :  77
user id completed :  456
user id completed :  276
user id completed :  232
user id completed :  165
user id completed :  3487
user id completed :  2022
user id completed :  11
user id completed :  63
user id completed :  122
user id completed :  3132
user id completed :  997
user id completed :  5043
user id completed :  633
user id completed :  477
user id completed :  323
user id completed :  1114
use

### Server Train Data Generation

In [12]:
# now time to write the server code ,same code as the user code, just a single folder server folder will be created inside the data/chicago2 folder and all the 77 r txts and 9 s txts will be written
import pandas as pd
import numpy as np
import math
import os
import shutil

from datetime import datetime, timedelta
crime_mapping = {
    'DECEPTIVE PRACTICE': 0,
    'THEFT': 1,
    'CRIMINAL DAMAGE': 2,
    'BATTERY': 3,
    'ROBBERY': 4,
    'ASSAULT': 5,
    'BURGLARY': 6,
    'NARCOTICS': 7
    }

side_neighbourhood_dict = {}
with open('./side_com_adj.txt', 'r') as f:
    # read the lines
    lines = f.readlines()
    # now here each line has 2 integers, 1st is side_id and 2nd is neighbourhood_id
    # create a dictionary, where key is neighbourhood_id and value is the side_id
    for line in lines:
        # split the line by space
        line = line.split()
        # get the side_id
        side_id = int(line[0])
        # get the neighbourhood_id
        neighbourhood_id = int(line[1])
        # modulus the side_id by 101
        side_id = side_id % 101
        # add the side_id to the dictionary
        side_neighbourhood_dict[neighbourhood_id-1] = side_id  # here 1 is reduced because id starts from 1, making it 0 based indexing

# dataset1 = pd.read_csv('training_crime_data_for_last_9_months.csv')
# test_start_date = dataset1['time'].iloc[0]

# START_DATE = datetime.strptime('2012-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# END_DATE = datetime.strptime('2013-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
START_DATE = datetime.strptime('04/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S')
END_DATE = datetime.strptime('12/31/2019 23:59:59', '%m/%d/%Y %H:%M:%S')

# print the number of days between this two dates including the start date
# print((END_DATE - START_DATE).days + 1)

# START_DATE = datetime.strptime('2012-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# END_DATE = datetime.strptime('2013-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# read the server short2 crime file
# server_crime_full_df = pd.read_csv('/content/drive/MyDrive/Undergrad_thesis_final/AIST_UCA/data_latlon_xy_with_neighbourhood_id.csv')
server_crime_full_df = pd.read_csv(f'./server_data_for_FED_AIST_{save_file_path}.csv')
# now filter the server crime full df so that it has uid only the filtered_uids_for_server_data
# server_crime_full_df = server_crime_full_df[server_crime_full_df['uid'].isin(filtered_uids_for_server_data)]

#take 90% data of the cross_mixing_df for server_cross_mixing_df
# server_cross_mixing_df = cross_mixing_df.sample(frac=90/100)

#rename the Date to time
# server_crime_full_df.rename(columns={'Date':'time'}, inplace=True)
# os.makedirs('/data/chicago2/server', exist_ok=True)

# portion_to_take_from_each_region = 0.7

# List of unique neighborhood IDs in the dataset
# unique_neighborhoods = server_crime_full_df['neighbourhood_id'].unique()

# Columns to include in the final dataset
# columns_to_include = ['time', 'IUCR', 'Primary Type', 'neighbourhood_id', 'lat', 'lon', 'TYPE_ID', 'uid']

# Initialize an empty DataFrame to store the final result
# final_server_dataset = pd.DataFrame(columns=columns_to_include)

# Loop through each neighborhood ID
# for neighborhood_id in unique_neighborhoods:
#     # Filter the data for the current neighborhood
#     neighborhood_data = server_crime_full_df[server_crime_full_df['neighbourhood_id'] == neighborhood_id]

#     # Calculate the number of samples to include (70%)
#     num_samples = int(len(neighborhood_data) * portion_to_take_from_each_region)

#     # Take the first 70% of the data
#     neighborhood_train_data = neighborhood_data.head(num_samples)
#     # print('neighbouhood train data type ',type(neighborhood_train_data))

#     # Verify that exactly 70% of the data is included
#     assert len(neighborhood_train_data) == num_samples, f"Verification failed for neighborhood_id {neighborhood_id}"
#     # print('type after including columns ',type(neighborhood_train_data[columns_to_include]))

#     # Append only the desired columns to the final dataset
#     # final_server_dataset = final_server_dataset.append(neighborhood_train_data[columns_to_include], ignore_index=True)
#     # Concatenate the neighborhood_train_data to the final dataset
#     final_server_dataset = pd.concat([final_server_dataset, neighborhood_train_data[columns_to_include]], ignore_index=True)

total_meighbors = 77
# total_days = 260
# total_days = 365 # counted from START_DATE and END_DATE, not 260
# total_days = 261 # counted from START_DATE and END_DATE,(END_DATE - START_DATE) (excluding the END_DATE) not 365
total_days = 275 # counted from START_DATE and END_DATE,(END_DATE - START_DATE) (including the END_DATE) not 261
time_division_per_day = 6
hours_per_cell  = 4
total_cols = total_days * time_division_per_day
# now we have only first 8 hours (07:59:59) for the END_DATE, so we cannot take the whole day, rather we will take additional(8/hours_per_cell) = (8/4)= 2 columns extra
# total_cols +=2
# total number of row is total crimes in crime_mapping
total_rows = len(crime_mapping)
# create a 3d tensor of dimension total_regions X crimes X total_cols
server_crime_tensor = np.zeros((total_meighbors, total_rows, total_cols))

'''code for adding the side crimes'''
# create a user side crime tensor of dimenstion 9 X crimes X total_cols
server_side_crime_tensor = np.zeros((9, total_rows, total_cols))

# Convert the time format to datetime
server_crime_full_df['time'] = pd.to_datetime(server_crime_full_df['time'], format='%Y-%m-%d %H:%M:%S')
# filter the df to have data from start date to 365 days
final_server_dataset = server_crime_full_df[(server_crime_full_df['time'] >= START_DATE) & (server_crime_full_df['time'] <= END_DATE)]

# run loop for each row
for index, row in final_server_dataset.iterrows():
    # get the neighbourhood id
    neighbourhood_id = row['neighbourhood_id']
    neighbourhood_id -=1 # for 0 based indexing
    # get the crime type id
    crime_type_id = row['TYPE_ID']
    # get the date
    date = row['time']
    # get the time
    time = date.time()
    # get the day
    day = date.date()
    # get the time division
    time_division = int(math.floor(time.hour / hours_per_cell))
    # get the day index
    day_index = (day - START_DATE.date()).days
    # get the time division index
    time_division_index = day_index * time_division_per_day + time_division
    # increment the value in the tensor
    server_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1    # for com_crime calculations
    cor_side_id = side_neighbourhood_dict[neighbourhood_id]
    server_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1    # for side_crime calculations

# Convert the time format to datetime
# server_cross_mixing_df['time'] = pd.to_datetime(server_cross_mixing_df['time'], format='%Y-%m-%d %H:%M:%S')
# filter the df to have data from start date to 365 days
# server_cross_mixing_df = server_cross_mixing_df[(server_cross_mixing_df['time'] >= START_DATE) & (server_cross_mixing_df['time'] <= END_DATE)]

# run loop for each row
# for index, row in server_cross_mixing_df.iterrows():
#     # get the neighbourhood id
#     neighbourhood_id = row['neighbourhood_id']
#     neighbourhood_id -=1 # for 0 based indexing
#     # get the crime type id
#     crime_type_id = row['TYPE_ID']
#     # get the date
#     date = row['time']
#     # get the time
#     time = date.time()
#     # get the day
#     day = date.date()
#     # get the time division
#     time_division = int(math.floor(time.hour / hours_per_cell))
#     # get the day index
#     day_index = (day - START_DATE.date()).days
#     # get the time division index
#     time_division_index = day_index * time_division_per_day + time_division
#     # increment the value in the tensor
#     server_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1
#     cor_side_id = side_neighbourhood_dict[neighbourhood_id]
#     server_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1


    # Iterate over the first dimension (total meighbors) and save each 2D slice as a text file
output_directory = f'./toy_data/chicago/server_{save_file_path}'
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

os.makedirs(output_directory)
for i in range(total_meighbors):
    filename = f"{output_directory}/r_{i}.txt"
    # Extract the 2D slice at index i
    slice_2d = server_crime_tensor[i, :, :]

    # Save the 2D slice as a text file
    np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")


# now same for side data
for i in range(9):
    filename = f"{output_directory}/s_{i}.txt"
    # Extract the 2D slice at index i
    slice_2d = server_side_crime_tensor[i, :, :]

    # Save the 2D slice as a text file
    np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")

print('here it ended')

here it ended


### Test Data Generation

In [ ]:
# now time to write the server code ,same code as the user code, just a single folder server folder will be created inside the data/chicago2 folder and all the 77 r txts and 9 s txts will be written
import pandas as pd
import numpy as np
import math
import os

from datetime import datetime, timedelta
crime_mapping = {
    'DECEPTIVE PRACTICE': 0,
    'THEFT': 1,
    'CRIMINAL DAMAGE': 2,
    'BATTERY': 3,
    'ROBBERY': 4,
    'ASSAULT': 5,
    'BURGLARY': 6,
    'NARCOTICS': 7
    }

side_neighbourhood_dict = {}
with open('./side_com_adj.txt', 'r') as f:
    # read the lines
    lines = f.readlines()
    # now here each line has 2 integers, 1st is side_id and 2nd is neighbourhood_id
    # create a dictionary, where key is neighbourhood_id and value is the side_id
    for line in lines:
        # split the line by space
        line = line.split()
        # get the side_id
        side_id = int(line[0])
        # get the neighbourhood_id
        neighbourhood_id = int(line[1])
        # modulus the side_id by 101
        side_id = side_id % 101
        # add the side_id to the dictionary
        side_neighbourhood_dict[neighbourhood_id-1] = side_id  # here 1 is reduced because id starts from 1, making it 0 based indexing

# START_DATE = datetime.strptime('2012-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# END_DATE = datetime.strptime('2013-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# START_DATE = datetime.strptime('09/19/2019 08:00:00', '%m/%d/%Y %H:%M:%S')
START_DATE = datetime.strptime('01/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S')
# END_DATE = datetime.strptime('12/31/2019 23:59:59', '%m/%d/%Y %H:%M:%S')
END_DATE = datetime.strptime('03/31/2019 23:59:59', '%m/%d/%Y %H:%M:%S')

# print the number of days between this two dates including the start date
# print((END_DATE - START_DATE).days + 1)

# START_DATE = datetime.strptime('2012-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# END_DATE = datetime.strptime('2013-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# read the server short2 crime file
# server_crime_full_df = pd.read_csv('/content/drive/MyDrive/Undergrad_thesis_final/AIST_UCA/data_latlon_xy_with_neighbourhood_id.csv')
test_crime_full_df = pd.read_csv('./testing_crime_data_for_first_3_months.csv')
# test_crime_full_df = test_crime_full_df.dropna()
# now filter the server crime full df so that it has uid only the filtered_uids_for_server_data
# server_crime_full_df = server_crime_full_df[server_crime_full_df['uid'].isin(filtered_uids_for_server_data)]

#take 90% data of the cross_mixing_df for server_cross_mixing_df
# server_cross_mixing_df = cross_mixing_df.sample(frac=90/100)

#rename the Date to time
# server_crime_full_df.rename(columns={'Date':'time'}, inplace=True)
# os.makedirs('/data/chicago2/server', exist_ok=True)

# portion_to_take_from_each_region = 0.7

# List of unique neighborhood IDs in the dataset
# unique_neighborhoods = server_crime_full_df['neighbourhood_id'].unique()

# Columns to include in the final dataset
# columns_to_include = ['time', 'IUCR', 'Primary Type', 'neighbourhood_id', 'lat', 'lon', 'TYPE_ID', 'uid']

# Initialize an empty DataFrame to store the final result
# final_server_dataset = pd.DataFrame(columns=columns_to_include)

# Loop through each neighborhood ID
# for neighborhood_id in unique_neighborhoods:
#     # Filter the data for the current neighborhood
#     neighborhood_data = server_crime_full_df[server_crime_full_df['neighbourhood_id'] == neighborhood_id]

#     # Calculate the number of samples to include (70%)
#     num_samples = int(len(neighborhood_data) * portion_to_take_from_each_region)

#     # Take the first 70% of the data
#     neighborhood_train_data = neighborhood_data.head(num_samples)
#     # print('neighbouhood train data type ',type(neighborhood_train_data))

#     # Verify that exactly 70% of the data is included
#     assert len(neighborhood_train_data) == num_samples, f"Verification failed for neighborhood_id {neighborhood_id}"
#     # print('type after including columns ',type(neighborhood_train_data[columns_to_include]))

#     # Append only the desired columns to the final dataset
#     # final_server_dataset = final_server_dataset.append(neighborhood_train_data[columns_to_include], ignore_index=True)
#     # Concatenate the neighborhood_train_data to the final dataset
#     final_server_dataset = pd.concat([final_server_dataset, neighborhood_train_data[columns_to_include]], ignore_index=True)

total_meighbors = 77
# total_days = 260
# total_days = 365 # counted from START_DATE and END_DATE, not 260
# total_days = 103 # counted from START_DATE and END_DATE,(END_DATE - START_DATE),
total_days = 90 # counted from START_DATE and END_DATE,(END_DATE - START_DATE),
#! (excluding the START_DATE, because the START_DATE has not the full 24 hrs, it has hours from 08:00:00) not 365
time_division_per_day = 6
hours_per_cell  = 4
total_cols = total_days * time_division_per_day
# total number of row is total crimes in crime_mapping
# now we have only last 16 hours (15:59:59 precisely) for the START_DATE because the time starts from 08:00:00, so we cannot take the whole day, rather we will take additional(16/hours_per_cell) = (16/4)= 4 columns extra
# total_cols += 4
total_rows = len(crime_mapping)
# create a 3d tensor of dimension total_regions X crimes X total_cols
server_crime_tensor = np.zeros((total_meighbors, total_rows, total_cols))

'''code for adding the side crimes'''
# create a user side crime tensor of dimenstion 9 X crimes X total_cols
server_side_crime_tensor = np.zeros((9, total_rows, total_cols))

# Convert the time format to datetime
test_crime_full_df['time'] = pd.to_datetime(test_crime_full_df['time'], format='%Y-%m-%d %H:%M:%S')
# filter the df to have data from start date to 365 days
# final_server_dataset = server_crime_full_df[(server_crime_full_df['time'] >= START_DATE) & (server_crime_full_df['time'] <= END_DATE)]

# run loop for each row
for index, row in test_crime_full_df.iterrows():
    # get the neighbourhood id
    neighbourhood_id = row['neighbourhood_id']
    neighbourhood_id -=1 # for 0 based indexing
    # get the crime type id
    crime_type_id = row['TYPE_ID']
    # get the date
    #! here we are subtracting 8 hours from the day because the START_DATE begins with time 08:00:00, we want to map 08:00:00 to 00:00:00, so we are subtracting 8 hours from the time slot for each row
    # date = row['time'] - timedelta(hours=8)
    date = row['time']
    # get the time
    time = date.time()
    # get the day
    day = date.date()
    # get the time division
    time_division = int(math.floor(time.hour / hours_per_cell))
    # get the day index
    day_index = (day - START_DATE.date()).days
    # get the time division index
    time_division_index = day_index * time_division_per_day + time_division
    # increment the value in the tensor
    server_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1    # for com_crime calculations
    cor_side_id = side_neighbourhood_dict[neighbourhood_id]
    server_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1    # for side_crime calculations

# Convert the time format to datetime
# server_cross_mixing_df['time'] = pd.to_datetime(server_cross_mixing_df['time'], format='%Y-%m-%d %H:%M:%S')
# filter the df to have data from start date to 365 days
# server_cross_mixing_df = server_cross_mixing_df[(server_cross_mixing_df['time'] >= START_DATE) & (server_cross_mixing_df['time'] <= END_DATE)]

# run loop for each row
# for index, row in server_cross_mixing_df.iterrows():
#     # get the neighbourhood id
#     neighbourhood_id = row['neighbourhood_id']
#     neighbourhood_id -=1 # for 0 based indexing
#     # get the crime type id
#     crime_type_id = row['TYPE_ID']
#     # get the date
#     date = row['time']
#     # get the time
#     time = date.time()
#     # get the day
#     day = date.date()
#     # get the time division
#     time_division = int(math.floor(time.hour / hours_per_cell))
#     # get the day index
#     day_index = (day - START_DATE.date()).days
#     # get the time division index
#     time_division_index = day_index * time_division_per_day + time_division
#     # increment the value in the tensor
#     server_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1
#     cor_side_id = side_neighbourhood_dict[neighbourhood_id]
#     server_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1


    # Iterate over the first dimension (total meighbors) and save each 2D slice as a text file
output_directory = "./toy_data/chicago/test"
for i in range(total_meighbors):
    filename = f"{output_directory}/r_{i}.txt"
    # Extract the 2D slice at index i
    slice_2d = server_crime_tensor[i, :, :]

    # Save the 2D slice as a text file
    np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")


# now same for side data
for i in range(9):
    filename = f"{output_directory}/s_{i}.txt"
    # Extract the 2D slice at index i
    slice_2d = server_side_crime_tensor[i, :, :]

    # Save the 2D slice as a text file
    np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")

print('here test data generation ended')

here test data generation ended


### Aggregated Data Generation For Central AIST

In [ ]:
# now time to write the server code ,same code as the user code, just a single folder server folder will be created inside the data/chicago2 folder and all the 77 r txts and 9 s txts will be written
import pandas as pd
import numpy as np
import math
import os

from datetime import datetime, timedelta
crime_mapping = {
    'DECEPTIVE PRACTICE': 0,
    'THEFT': 1,
    'CRIMINAL DAMAGE': 2,
    'BATTERY': 3,
    'ROBBERY': 4,
    'ASSAULT': 5,
    'BURGLARY': 6,
    'NARCOTICS': 7
    }

side_neighbourhood_dict = {}
with open('./side_com_adj.txt', 'r') as f:
    # read the lines
    lines = f.readlines()
    # now here each line has 2 integers, 1st is side_id and 2nd is neighbourhood_id
    # create a dictionary, where key is neighbourhood_id and value is the side_id
    for line in lines:
        # split the line by space
        line = line.split()
        # get the side_id
        side_id = int(line[0])
        # get the neighbourhood_id
        neighbourhood_id = int(line[1])
        # modulus the side_id by 101
        side_id = side_id % 101
        # add the side_id to the dictionary
        side_neighbourhood_dict[neighbourhood_id-1] = side_id  # here 1 is reduced because id starts from 1, making it 0 based indexing

# START_DATE = datetime.strptime('2012-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# END_DATE = datetime.strptime('2013-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# START_DATE = datetime.strptime('01/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S')
# END_DATE = datetime.strptime('09/19/2019 07:59:59', '%m/%d/%Y %H:%M:%S')

START_DATE = datetime.strptime('04/01/2019 00:00:00', '%m/%d/%Y %H:%M:%S')
END_DATE = datetime.strptime('12/31/2019 23:59:59', '%m/%d/%Y %H:%M:%S')

# print the number of days between this two dates including the start date
# print((END_DATE - START_DATE).days + 1)

# START_DATE = datetime.strptime('2012-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# END_DATE = datetime.strptime('2013-04-3 00:00:00', '%Y-%m-%d %H:%M:%S')
# read the server short2 crime file
# server_crime_full_df = pd.read_csv('/content/drive/MyDrive/Undergrad_thesis_final/AIST_UCA/data_latlon_xy_with_neighbourhood_id.csv')
# full_crime_full_df = pd.read_csv('./training_crime_data_for_last_9_months.csv')
full_crime_full_df = pd.read_csv('./Final_Training_Data.csv')
# full_crime_full_df = full_crime_full_df.dropna()
full_crime_full_df.rename(columns={"Community Area": "neighbourhood_id"}, inplace=True)

# now filter the server crime full df so that it has uid only the filtered_uids_for_server_data
# server_crime_full_df = server_crime_full_df[server_crime_full_df['uid'].isin(filtered_uids_for_server_data)]

#take 90% data of the cross_mixing_df for server_cross_mixing_df
# server_cross_mixing_df = cross_mixing_df.sample(frac=90/100)

#rename the Date to time
# server_crime_full_df.rename(columns={'Date':'time'}, inplace=True)
# os.makedirs('/data/chicago2/server', exist_ok=True)

# portion_to_take_from_each_region = 0.7

# List of unique neighborhood IDs in the dataset
# unique_neighborhoods = server_crime_full_df['neighbourhood_id'].unique()

# Columns to include in the final dataset
# columns_to_include = ['time', 'IUCR', 'Primary Type', 'neighbourhood_id', 'lat', 'lon', 'TYPE_ID', 'uid']

# Initialize an empty DataFrame to store the final result
# final_server_dataset = pd.DataFrame(columns=columns_to_include)

# Loop through each neighborhood ID
# for neighborhood_id in unique_neighborhoods:
#     # Filter the data for the current neighborhood
#     neighborhood_data = server_crime_full_df[server_crime_full_df['neighbourhood_id'] == neighborhood_id]

#     # Calculate the number of samples to include (70%)
#     num_samples = int(len(neighborhood_data) * portion_to_take_from_each_region)

#     # Take the first 70% of the data
#     neighborhood_train_data = neighborhood_data.head(num_samples)
#     # print('neighbouhood train data type ',type(neighborhood_train_data))

#     # Verify that exactly 70% of the data is included
#     assert len(neighborhood_train_data) == num_samples, f"Verification failed for neighborhood_id {neighborhood_id}"
#     # print('type after including columns ',type(neighborhood_train_data[columns_to_include]))

#     # Append only the desired columns to the final dataset
#     # final_server_dataset = final_server_dataset.append(neighborhood_train_data[columns_to_include], ignore_index=True)
#     # Concatenate the neighborhood_train_data to the final dataset
#     final_server_dataset = pd.concat([final_server_dataset, neighborhood_train_data[columns_to_include]], ignore_index=True)

total_meighbors = 77
# total_days = 260
# total_days = 365 # counted from START_DATE and END_DATE, not 260
# total_days = 261 # counted from START_DATE and END_DATE,(END_DATE - START_DATE), excluding the END_DATE because we dont have whole 24 hours for the END_DATE not 365
total_days = 275 # counted from START_DATE and END_DATE,(END_DATE - START_DATE), including the END_DATE because we dont have whole 24 hours for the END_DATE not 365
time_division_per_day = 6
hours_per_cell  = 4
total_cols = total_days * time_division_per_day
# now we have only first 8 hours (07:59:59) for the last date, so we cannot take the whole day, rather we will take additional(8/hours_per_cell) = (8/4)= 2 columns extra
# total_cols +=2
# total number of row is total crimes in crime_mapping
total_rows = len(crime_mapping)
# create a 3d tensor of dimension total_regions X crimes X total_cols
server_crime_tensor = np.zeros((total_meighbors, total_rows, total_cols))

'''code for adding the side crimes'''
# create a user side crime tensor of dimenstion 9 X crimes X total_cols
server_side_crime_tensor = np.zeros((9, total_rows, total_cols))

# Convert the time format to datetime
full_crime_full_df['time'] = pd.to_datetime(full_crime_full_df['time'], format='%Y-%m-%d %H:%M:%S')
# filter the df to have data from start date to 365 days
# final_server_dataset = server_crime_full_df[(server_crime_full_df['time'] >= START_DATE) & (server_crime_full_df['time'] <= END_DATE)]

# run loop for each row
for index, row in full_crime_full_df.iterrows():
    # get the neighbourhood id
    neighbourhood_id = row['neighbourhood_id']
    neighbourhood_id -=1 # for 0 based indexing
    # get the crime type id
    crime_type_id = row['TYPE_ID']
    # get the date
    date = row['time']
    # get the time
    time = date.time()
    # get the day
    day = date.date()
    # get the time division
    time_division = int(math.floor(time.hour / hours_per_cell))
    # get the day index
    day_index = (day - START_DATE.date()).days
    # get the time division index
    time_division_index = day_index * time_division_per_day + time_division
    # increment the value in the tensor
    server_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1    # for com_crime calculations
    cor_side_id = side_neighbourhood_dict[neighbourhood_id]
    server_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1    # for side_crime calculations

# Convert the time format to datetime
# server_cross_mixing_df['time'] = pd.to_datetime(server_cross_mixing_df['time'], format='%Y-%m-%d %H:%M:%S')
# filter the df to have data from start date to 365 days
# server_cross_mixing_df = server_cross_mixing_df[(server_cross_mixing_df['time'] >= START_DATE) & (server_cross_mixing_df['time'] <= END_DATE)]

# run loop for each row
# for index, row in server_cross_mixing_df.iterrows():
#     # get the neighbourhood id
#     neighbourhood_id = row['neighbourhood_id']
#     neighbourhood_id -=1 # for 0 based indexing
#     # get the crime type id
#     crime_type_id = row['TYPE_ID']
#     # get the date
#     date = row['time']
#     # get the time
#     time = date.time()
#     # get the day
#     day = date.date()
#     # get the time division
#     time_division = int(math.floor(time.hour / hours_per_cell))
#     # get the day index
#     day_index = (day - START_DATE.date()).days
#     # get the time division index
#     time_division_index = day_index * time_division_per_day + time_division
#     # increment the value in the tensor
#     server_crime_tensor[neighbourhood_id][crime_type_id][time_division_index] += 1
#     cor_side_id = side_neighbourhood_dict[neighbourhood_id]
#     server_side_crime_tensor[cor_side_id][crime_type_id][time_division_index] += 1


    # Iterate over the first dimension (total meighbors) and save each 2D slice as a text file
output_directory = "./toy_data/chicago/aggregated"
for i in range(total_meighbors):
    filename = f"{output_directory}/r_{i}.txt"
    # Extract the 2D slice at index i
    slice_2d = server_crime_tensor[i, :, :]

    # Save the 2D slice as a text file
    np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")


# now same for side data
for i in range(9):
    filename = f"{output_directory}/s_{i}.txt"
    # Extract the 2D slice at index i
    slice_2d = server_side_crime_tensor[i, :, :]

    # Save the 2D slice as a text file
    np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")

print('here aggregated data generation ended')

here aggregated data generation ended


# Unused

In [ ]:
# taxi_trip_df = pd.read_csv('/data/chicago/out_Data.csv')
# # drop the empty rows
# taxi_trip_df = taxi_trip_df.dropna()
# # print the shape
# print(taxi_trip_df.shape)

In [ ]:
# # here taxi data set creation code
# # this calc is for "taxi in data"
# from datetime import datetime, timedelta

# new_start_date = datetime.strptime('2013-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
# new_end_date =  datetime.strptime('2013-09-16 23:59:59', '%Y-%m-%d %H:%M:%S')

# # read the Taxi trip csv
# # ----------------------------------------
# taxi_trip_df = pd.read_csv('/data/chicago/out_Data.csv')
# # drop the empty rows
# taxi_trip_df = taxi_trip_df.dropna()
# #-------------------------------------------
# # filter the df to have data from new start date to end date
# taxi_trip_df = taxi_trip_df[(taxi_trip_df['Trip Start Timestamp'] >= new_start_date) & (taxi_trip_df['Trip Start Timestamp'] <= new_end_date)]
# # rename the start time stamp column to start time and end timestamp column to end time, Pickup Community Area to out , Dropoff Community Area to in
# taxi_trip_df.rename(columns={'Trip Start Timestamp':'out_time', 'Pickup Community Area':'out'}, inplace=True)
# # get the number of days
# num_days = (new_end_date - new_start_date).days + 1 # 260
# # get the number of time divisions
# num_time_divisions = 2  # here time divison is 2 means each day has 2 time slots
# # get the number of regions
# num_regions = 77
# # get the number of rows
# num_rows = 2
# # get the number of columns
# num_cols = num_days * num_time_divisions # 260 * 2
# # create a 3d tensor of dimension num_regions X num_rows X num_cols
# taxi_trip_tensor = np.zeros((num_regions, num_rows, num_cols))
# # Convert the time format to datetime
# taxi_trip_df['out_time'] = pd.to_datetime(taxi_trip_df['out_time'], format='%m/%d/%Y %I:%M:%S %p')
# # run loop for each row
# for index, row in taxi_trip_df.iterrows():
#     # check if out and in times are within range
#     if row['out_time'] >= new_start_date and row['out_time'] <= new_end_date:
#         # get the out region
#         out_region = row['out']
#         # get the out time
#         out_time = row['out_time']
#         # get the out day
#         out_day = out_time.date()
#         # get the out time division
#         out_time_division = int(math.floor(out_time.hour / 12))  # here each day is divided in 12 hrs time slot,

#         # get the out day index
#         out_day_index = (out_day - new_start_date.date()).days
#         # get the out time division index
#         out_time_division_index = out_day_index * num_time_divisions + out_time_division
#         # get the in time division index
#         # increment the value in the tensor
#         taxi_trip_tensor[out_region - 1][1][out_time_division_index] += 1

In [ ]:
# # here taxi data set creation code
# # this calc is for "taxi in data"
# from datetime import datetime, timedelta

# new_start_date = datetime.strptime('2013-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
# new_end_date =  datetime.strptime('2013-09-16 23:59:59', '%Y-%m-%d %H:%M:%S')

# # read the Taxi trip csv
# # ----------------------------------------
# taxi_trip_df = pd.read_csv('/data/chicago/in_Data.csv')
# # drop the empty rows
# taxi_trip_df = taxi_trip_df.dropna()
# #-------------------------------------------
# # filter the df to have data from new start date to end date
# taxi_trip_df = taxi_trip_df[(taxi_trip_df['Trip End Timestamp'] >= new_start_date) & (taxi_trip_df['Trip End Timestamp'] <= new_end_date)]
# # rename the start time stamp column to start time and end timestamp column to end time, Pickup Community Area to out , Dropoff Community Area to in
# taxi_trip_df.rename(columns={'Trip End Timestamp':'in_time', 'Dropoff Community Area':'in'}, inplace=True)

# # Convert the time format to datetime
# taxi_trip_df['in_time'] = pd.to_datetime(taxi_trip_df['in_time'], format='%m/%d/%Y %I:%M:%S %p')
# # run loop for each row
# for index, row in taxi_trip_df.iterrows():
#     # check if in and in times are within range
#     if row['in_time'] >= new_start_date and row['in_time'] <= new_end_date:
#         # get the in region
#         in_region = row['in']
#         # get the in time
#         in_time = row['in_time']
#         # get the in day
#         in_day = in_time.date()
#         # get the in time division
#         in_time_division = int(math.floor(in_time.hour / 12)) # here dividing the day in 12 hrs, so each day has two slots indexed 0,1

#         # get the in day index
#         in_day_index = (in_day - new_start_date.date()).days
#         # get the in time division index
#         in_time_division_index = in_day_index * num_time_divisions + in_time_division
#         # get the in time division index
#         # increment the value in the tensor
#         taxi_trip_tensor[in_region - 1][0][in_time_division_index] += 1

In [ ]:
# # for i in range(77):
# #     file = open('content/drive/MyDrive/Undergrad_Thesis_Group_2_FedCrime/AIST/data/chicago2/act_ext' + '/taxi_' + str(i+1) + '.txt', 'w')
# #     for j in range(total_rows):
# #         for k in range(total_cols):
# #             file.write(str(int(server_side_crime_tensor[i][j][k])) + " ")
# #         file.write("\n")
# #     file.close()

# output_directory = "/data/chicago/act_ext"
# for i in range(77):
#     filename = f"{output_directory}/taxi_{i+1}.txt"
#     # Extract the 2D slice at index i
#     slice_2d = taxi_trip_tensor[i, :, :]

#     # Save the 2D slice as a text file
#     np.savetxt(filename, slice_2d, fmt="%d", delimiter=" ")